In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import DenseNet121
from keras.models import Model
from keras.layers import Dense, Flatten, BatchNormalization, GlobalAveragePooling2D, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import resample

#Đường dẫn thư mục train và test
train_path = r'.\train'
test_path = r'.\test'
val_path = r'.\val'

batch_size = 16 

img_height = 500
img_width = 500

In [2]:
image_gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.8,1.2]
)

# Data augmentation for testing/validation
test_data_gen = ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = image_gen.flow_from_directory(
    train_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb'
)

# Testing data generator
test = test_data_gen.flow_from_directory(
    test_path,
    target_size=(img_height, img_width),
    color_mode='rgb', 
    shuffle=False,
    class_mode='binary',
    batch_size=batch_size
)

# Validation data generator
validation_generator = image_gen.flow_from_directory(
    val_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    color_mode='rgb'
)

Found 1943 images belonging to 2 classes.
Found 611 images belonging to 2 classes.
Found 497 images belonging to 2 classes.


In [4]:
X_train, y_train = [], []
for _ in range(train_generator.n // batch_size + 1):
    x, y = train_generator.next()
    X_train.append(x)
    y_train.append(y)

X_train = np.vstack(X_train)
y_train = np.hstack(y_train)

# Separate majority and minority classes
normal = X_train[y_train == 0]
disease = X_train[y_train == 1]

# Oversample minority class
normal_upsampled = resample(normal, 
                            replace=True,     # sample with replacement
                            n_samples=len(disease),    # to match majority class
                            random_state=123) # reproducible results

# Combine majority class with upsampled minority class
X_train_balanced = np.vstack([normal_upsampled, disease])
y_train_balanced = np.hstack([np.zeros(len(disease)), np.ones(len(disease))])

# Shuffle the dataset
indices = np.arange(X_train_balanced.shape[0])
np.random.shuffle(indices)
X_train_balanced = X_train_balanced[indices]
y_train_balanced = y_train_balanced[indices]

In [5]:
base_model = DenseNet121(include_top=False, input_shape=(img_height, img_width, 3))
for layer in base_model.layers[:-4]:  # Fine-tune the last 4 layers
    layer.trainable = False

# Add custom layers on top of the pre-trained model
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Add dropout layer
x = Dense(64, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)  # Add dropout layer
predictions = Dense(1, activation='sigmoid')(x)

# Create the model
cnn = Model(inputs=base_model.input, outputs=predictions)
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.3, min_lr=0.000001)
callbacks_list = [early, learning_rate_reduction]

# Train the model
cnn.fit(
    X_train_balanced, y_train_balanced,
    epochs=50,  # Increase the number of epochs
    validation_data=validation_generator,
    callbacks=callbacks_list,
    batch_size=batch_size
)


29084464/29084464 [==============================] - 4s 0us/step
Epoch 1/50
69/69 [==============================] - 320s 5s/step - loss: 0.9209 - accuracy: 0.5499 - val_loss: 0.7362 - val_accuracy: 0.4748 - lr: 0.0010
Epoch 2/50
69/69 [==============================] - 309s 4s/step - loss: 0.7433 - accuracy: 0.6198 - val_loss: 0.7582 - val_accuracy: 0.5070 - lr: 0.0010
Epoch 3/50
69/69 [==============================] - 326s 5s/step - loss: 0.6337 - accuracy: 0.6887 - val_loss: 0.6333 - val_accuracy: 0.6841 - lr: 0.0010
Epoch 4/50
69/69 [==============================] - 316s 5s/step - loss: 0.6566 - accuracy: 0.6579 - val_loss: 0.7667 - val_accuracy: 0.5634 - lr: 0.0010
Epoch 5/50
69/69 [==============================] - 318s 5s/step - loss: 0.5949 - accuracy: 0.6996 - val_loss: 0.8248 - val_accuracy: 0.5674 - lr: 0.0010
Epoch 6/50
69/69 [==============================] - ETA: 0s - loss: 0.5915 - accuracy: 0.7142
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0003000000142492

In [6]:
cnn.save('DenseNet1.h5')

c:\Program Files\Python38\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
from sklearn.metrics import confusion_matrix, classification_report

# Evaluate the model on the test data
test_accu = cnn.evaluate(test)
print('The testing accuracy is:', test_accu[1] * 100, '%')

# Predict the labels for the test data
y_pred = cnn.predict(test)
y_pred = np.round(y_pred).astype(int)  # Chuyển đổi dự đoán thành nhãn nhị phân (0 hoặc 1)

# Get the true labels
y_true = test.classes

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Compute classification report
class_report = classification_report(y_true, y_pred, target_names=['Normal', 'Disease'])
print('Classification Report:')
print(class_report)

# Extract Sensitivity, Specificity, and F1-Score from confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn)
specificity = tn / (tn + fp)
precision = tp / (tp + fp)
recall = sensitivity  # Recall là tên khác của Sensitivity
f1_score = 2 * (precision * recall) / (precision + recall)

print(f'Sensitivity: {sensitivity * 100:.2f}%')
print(f'Specificity: {specificity * 100:.2f}%')
print(f'F1-Score: {f1_score:.2f}')

39/39 [==============================] - 120s 3s/step - loss: 0.5105 - accuracy: 0.7692
The testing accuracy is: 76.92307829856873 %
39/39 [==============================] - 133s 3s/step
Confusion Matrix:
[[319  95]
 [ 46 151]]
Classification Report:
              precision    recall  f1-score   support

      Normal       0.87      0.77      0.82       414
     Disease       0.61      0.77      0.68       197

    accuracy                           0.77       611
   macro avg       0.74      0.77      0.75       611
weighted avg       0.79      0.77      0.77       611

Sensitivity: 76.65%
Specificity: 77.05%
F1-Score: 0.68
